In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def get_recommendations(user_keywords, tfidf_vectorizer, tfidf_matrix, food_data, user_calories_threshold):
    keywords = user_keywords[0]
    # 사용자 키워드를 조회할 때 공백(' ')에 조인하여 하나의 문자열로 처리
    # 사용자 키워드에 대한 TF-IDF 행렬 생성
    user_tfidf_matrix = tfidf_vectorizer.transform([' '.join(keywords)])
    
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 음식 추천 알고리즘
    # 코사인 유사도 행렬에서 첫 번째 행(사용자 입력과 음식 간의 유사도)을 가져와서
    # 각 음식과의 유사도를 인덱스와 함께 리스트로 변환 => 각 키워드가 가진 유사도를 음식과 전체 조회하여 반환
    sim_scores = list(enumerate(cosine_sim[0]))
   
    # 유사도를 기준으로 리스트를 내림차순으로 정렬
    # key=lambda x: x[1]는 리스트의 각 요소를 유사도에 대해 정렬하기 위한 키 함수
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 가장 높은 음식(sim_scores[0])을 제외하고 반환
    # 가장 높은 음식을 반환하는 이유는 검색기록이 1개인 경우 그 데이터가 가장 높은 유사도를 가지게 되므로
    sim_scores = sim_scores[1:]

    # sim_scores에 저장된 음식들의 인덱스를 저장
    # sim_scores에 저장된 형태 => (인덱스 번호, 유사도 점수)
    food_indices = [idx[0] for idx in sim_scores]

    # sim_scores에 저장된 음식에 대하여 iloc(food_indices)를 통하여 이름 조회
    cosine_similar_food = food_data.iloc[food_indices][['Name','Calories']]
    cosine_similar_food = cosine_similar_food[cosine_similar_food['Calories'] < user_calories_threshold]
    return cosine_similar_food

def user_required_calories(user_height, user_weight, user_age, user_gender):
    height_m = user_height / 100
    weight_kg = user_weight
    bmi = weight_kg / (height_m ** 2)

    if user_gender == 'Male':
        bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_m * 100) - (5.677 * user_age)  # 테스트를 위해 임의의 나이 사용
    else:
        bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_m * 100) - (4.330 * user_age)

    calories_threshold = bmr * 1.2  # 활동 수준을 고려하여 칼로리 요구량을 조정(1.2로 고정[운동x])

    return calories_threshold

def weighted_nutrition(df, nutrition):
    percentile = 0.6
    m = df[nutrition].quantile(percentile)
    C = df['Calories'].mean()
    v = df[nutrition]
    R = df['Calories']

    return (((v / (v+m)) * R) + ((m / (v+m)) * C)) / R

def recommend_similar_food(similarities, food, user_calories_threshold):
    recommended_foods = []
    for user, sim_scores in similarities.items():
        # 음식과 사용자의 선호도 간의 코사인 유사도 계산
        # 유사도가 가장 높은 상위 30개 인덱스 추출
        recommended_food_indices = np.argsort(sim_scores)[-30:][::-1]
        # 추천된 음식들의 이름과 인덱스, 칼로리 저장
        recommended_foods.extend([(idx, food.iloc[idx]['Name'], food.iloc[idx]['Calories']) for idx in recommended_food_indices])
    # DataFrame으로 변환
    recommended_foods_df = pd.DataFrame(recommended_foods, columns=['Index', 'Name', 'Calories'])
    # 사용자의 칼로리 요구량보다 낮은 음식만 필터링
    recommended_foods_df = recommended_foods_df[recommended_foods_df['Calories'] < user_calories_threshold]
    return recommended_foods_df

# 예시) def test1(food, max_calories, excluded_ingredients):
    

# 음식 데이터 로드
# 음식 데이터 로드 시 'Calories' 열의 데이터 형식을 숫자(float)로 지정하여 로드
food = pd.read_csv('./foodData4_24.csv', dtype={'Calories': float})

# 결측값 처리(Keywords열의 데이터 중 nan값 공백으로 처리)
# 현재 food 데이터에 일부 깨진데이터(?)가 있어서 결측값 처리해야 정상작동
food['Keywords'] = food['Keywords'].fillna('')

# TF-IDF(Term Frequency-Inverse Document Frequency)
# 텍스트 데이터의 통계적인 가중치를 계산, 처리해주는 기능(자연어 처리)

# TfidfVectorizer : 문서를 벡터 표현으로 바꿔주는 기능
# stop_words='english' => 영어의 일반적인 불용어 삭제(and, is, the, this 등 단어와 상관없는 문자)
tfidf = TfidfVectorizer(stop_words='english')

# food의 Keywords를 단어로 구별
# 단어로 구별된 Keywords를 TF-IDF 행렬로 생성
tfidf_matrix = tfidf.fit_transform(food['Keywords'])

# 각 영양소에 대한 가중평균 계산
food['Carbohydrate_weighted_rate'] = weighted_nutrition(food, 'CarbohydrateContent')
food['Protein_weighted_rate'] = weighted_nutrition(food, 'ProteinContent')
food['Fat_weighted_rate'] = weighted_nutrition(food, 'FatContent')

# 사용자1의 예제 데이터
user1 = [['Dessert', 'Pork', 'Grains', 'Oven', 'Christmas', 'Brunch'],[user_required_calories(178.9, 72.3, 24, 'Male')/3]]

# 음식 추천
user_calories_threshold = user1[1][0]  # user1[1]은 리스트이므로 첫 번째 요소를 가져옴
## 예시) food = test1(food, 300, ['carrot'])
similarities = get_recommendations(user1, tfidf, tfidf_matrix, food, user_calories_threshold)
print(similarities['Name'])
recommendations = recommend_similar_food(similarities, food, user_calories_threshold)

print("추천 음식:")
print(recommendations)
# 30개의 인덱스 추출을 했는데 왜 60개지??????

142321                        Coffee Chocolate Waffles
66308     Tina's Overnight Sausage &amp; Egg Casserole
15217                                          "Tomato
6498                      Very Soft Rolled Oat Cookies
14133                     Peggy's Foolproof  Pie Crust
                              ...                     
165885                   Masala Maggi Noodles in a Mug
165886                         Chocolate Rum Snowballs
165891              Fun & Delicious Homemade Pop Tarts
165893                   Spanish Coffee with Tia Maria
165896        Quick & Easy Asian Cucumber Salmon Rolls
Name: Name, Length: 144031, dtype: object
추천 음식:
     Index                                               Name  Calories
1    45446                Delicious Cranberry-Pineapple Sauce     273.8
2    41592                           Peanut Butter Fudge Cups      80.9
3    21908                              Banana Berry Smoothie     288.1
4    45178                     Colored Striped Icebox Cook

# 기본 코드 백업

In [4]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer

def get_recommendations(user_keywords, tfidf_vectorizer, tfidf_matrix, food_data):
    keywords = user_keywords[0]
    # 사용자 키워드를 조회할 때 공백(' ')에 조인하여 하나의 문자열로 처리
    # 사용자 키워드에 대한 TF-IDF 행렬 생성
    user_tfidf_matrix = tfidf_vectorizer.transform([' '.join(keywords)])
    
    # 사용자 키워드 행렬과 food 데이터의 키워드 행렬의 코사인 유사도 계산
    cosine_sim = cosine_similarity(user_tfidf_matrix, tfidf_matrix)
   
    # 음식 추천 알고리즘
    # 코사인 유사도 행렬에서 첫 번째 행(사용자 입력과 음식 간의 유사도)을 가져와서
    # 각 음식과의 유사도를 인덱스와 함께 리스트로 변환 => 각 키워드가 가진 유사도를 음식과 전체 조회하여 반환
    sim_scores = list(enumerate(cosine_sim[0]))
   
    # 유사도를 기준으로 리스트를 내림차순으로 정렬
    # key=lambda x: x[1]는 리스트의 각 요소를 유사도에 대해 정렬하기 위한 키 함수
    sim_scores = sorted(sim_scores, key=lambda x: x[1], reverse=True)
    
    # 유사도가 가장 높은 음식(sim_scores[0])을 제외하고 반환
    # 가장 높은 음식을 반환하는 이유는 검색기록이 1개인 경우 그 데이터가 가장 높은 유사도를 가지게 되므로
    sim_scores = sim_scores[1:]

    # sim_scores에 저장된 음식들의 인덱스를 저장
    # sim_scores에 저장된 형태 => (인덱스 번호, 유사도 점수)
    food_indices = [idx[0] for idx in sim_scores]

    # sim_scores에 저장된 음식에 대하여 iloc(food_indices)를 통하여 이름 조회
    cosine_similar_food = food_data.iloc[food_indices][['Name','Calories']]
    return cosine_similar_food

def user_required_calories(user_height, user_weight, user_age, user_gender):
    height_m = user_height / 100
    weight_kg = user_weight
    bmi = weight_kg / (height_m ** 2)

    if user_gender == 'Male':
        bmr = 88.362 + (13.397 * weight_kg) + (4.799 * height_m * 100) - (5.677 * user_age)  # 테스트를 위해 임의의 나이 사용
    else:
        bmr = 447.593 + (9.247 * weight_kg) + (3.098 * height_m * 100) - (4.330 * user_age)

    calories_threshold = bmr * 1.2  # 활동 수준을 고려하여 칼로리 요구량을 조정(1.2로 고정[운동x])

    return calories_threshold

def weighted_nutrition(df, nutrition):
    percentile = 0.6
    m = df[nutrition].quantile(percentile)
    C = df['Calories'].mean()
    v = df[nutrition]
    R = df['Calories']

    return (((v / (v+m)) * R) + ((m / (v+m)) * C)) / R

def recommend_similar_food(similarities, food):
    recommended_foods = []
    for user, sim_scores in similarities.items():
        # 음식과 사용자의 선호도 간의 코사인 유사도 계산
        # 유사도가 가장 높은 상위 30개 인덱스 추출
        recommended_food_indices = np.argsort(sim_scores)[:30][::-1]
        # 추천된 음식들의 이름과 인덱스 저장
        recommended_foods.extend([(idx, food.iloc[idx]['Name'], food.iloc[idx]['Calories']) for idx in recommended_food_indices])
    return recommended_foods

# 음식 데이터 로드
# 음식 데이터 로드 시 'Calories' 열의 데이터 형식을 숫자(float)로 지정하여 로드
food = pd.read_csv('./foodData4_24.csv', dtype={'Calories': float})

# 결측값 처리(Keywords열의 데이터 중 nan값 공백으로 처리)
# 현재 food 데이터에 일부 깨진데이터(?)가 있어서 결측값 처리해야 정상작동
food['Keywords'] = food['Keywords'].fillna('')

# TF-IDF(Term Frequency-Inverse Document Frequency)
# 텍스트 데이터의 통계적인 가중치를 계산, 처리해주는 기능(자연어 처리)

# TfidfVectorizer : 문서를 벡터 표현으로 바꿔주는 기능
# stop_words='english' => 영어의 일반적인 불용어 삭제(and, is, the, this 등 단어와 상관없는 문자)
tfidf = TfidfVectorizer(stop_words='english')

# food의 Keywords를 단어로 구별
# 단어로 구별된 Keywords를 TF-IDF 행렬로 생성
tfidf_matrix = tfidf.fit_transform(food['Keywords'])

# 각 영양소에 대한 가중평균 계산
food['Carbohydrate_weighted_rate'] = weighted_nutrition(food, 'CarbohydrateContent')
food['Protein_weighted_rate'] = weighted_nutrition(food, 'ProteinContent')
food['Fat_weighted_rate'] = weighted_nutrition(food, 'FatContent')

# 사용자1의 예제 데이터
user1 = [['Dessert', 'Pork', 'Grains', 'Oven', 'Christmas', 'Brunch'],[user_required_calories(178.9, 72.3, 24, 'Male')/3]]

# 음식 추천
similarities = get_recommendations(user1, tfidf, tfidf_matrix, food)
recommendations = recommend_similar_food(similarities, food)

print("추천 음식:")
print(recommendations)

추천 음식:
[(2335, 'Grilled Chicken Dijon', 148.4), (52469, 'Hearty Oatmeal Loaf--Abm', 148.1), (58236, "Pat's Gourmet Salmon Cakes", 1418.1), (14831, 'Chinese Green Beans', 32.8), (51505, "Bobby Flay's Quick Saffron Risotto With Roasted Asparagus", 530.2), (59772, 'Asparagus With Sesame Oil Vinaigrette', 116.8), (11399, 'EZ Cheesy Potatoes', 164.0), (21925, 'Scotch Eggs, Baked Not Fried!', 388.0), (46729, 'Jungle Colada from Elephant Bar', 422.2), (43742, "Paula Deen's 5 Minute Fudge", 185.5), (61810, "Threadgill's Garlic Cheese Grits", 173.6), (55536, 'Raspberry Dressing (Vinaigrette), No Added Sugar', 160.4), (61122, 'Cheesy Bean Dip', 380.0), (28447, 'Berry Lemon Mousse Parfait', 223.1), (44867, 'Peachy Chicken', 477.0), (54859, 'Healthy Warm Artichoke Dip', 167.1), (33601, 'Simple Pizza Dough', 187.3), (53774, 'Portabella Mushroom Bruschetta', 262.1), (60102, "Rogene's Chili Sauce Meatballs", 276.0), (89344, 'Best Beef Tips and Gravy', 884.5), (46431, 'Lemon Blossoms (Paula Deen)', 11

In [5]:
print([food['Carbohydrate_weighted_rate'], food['Protein_weighted_rate'], food['Fat_weighted_rate']])

[0          1.785781
1          0.828692
2          1.132800
3          0.944742
4          2.933194
            ...    
165892     6.380114
165893     3.927443
165894     0.901659
165895     0.285373
165896    27.769353
Name: Carbohydrate_weighted_rate, Length: 165897, dtype: float64, 0          2.314599
1          0.898915
2          1.439134
3          0.951139
4          3.518844
            ...    
165892     6.939733
165893     5.156525
165894     0.727965
165895     0.921851
165896    23.795729
Name: Protein_weighted_rate, Length: 165897, dtype: float64, 0          2.428168
1          0.866765
2          1.444044
3          0.934079
4          4.274942
            ...    
165892     5.936256
165893     4.870272
165894     0.836553
165895     0.929855
165896    27.083575
Name: Fat_weighted_rate, Length: 165897, dtype: float64]


In [11]:
# inf 값을 제외한 데이터 추출
filtered_data = food['Carbohydrate_weighted_rate'][food['Carbohydrate_weighted_rate'] != float('inf')]

# inf 값을 제외한 데이터 중에서 가장 큰 값 구하기
max_without_inf = filtered_data.max()

print(max_without_inf)

4508.671615520473
